In [2]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [3]:
from fastai.vision.all import *
from tqdm.auto import tqdm
import gc
import torchvision.transforms.functional as TF
import random

In [7]:
# sys.path.append('../')
# import warnings; warnings.filterwarnings('ignore')

In [5]:
from src.defaults import get_cfg
from src.dataset import LoadDatasets, get_class_names
from src.models import build_model
from timm.models import load_checkpoint

In [6]:
CLASS_NAMES = get_class_names(nc=4)
CLASS_NAMES

In [10]:
#### YOUR BEST MODELS
CHECKPOINTS = [
    '/home/shivgup/siim/heng/code/ayush/new_code/code/runs/001cf4cc-8939v2l_noisy_Fine/fold_0/model_best.pth.tar',
    '/home/shivgup/siim/heng/code/ayush/new_code/code/runs/001cf4cc-8939v2l_noisy_Fine/fold_1/model_best.pth.tar',
    '/home/shivgup/siim/heng/code/ayush/new_code/code/runs/001cf4cc-8939v2l_noisy_Fine/fold_2/model_best.pth.tar',
    '/home/shivgup/siim/heng/code/ayush/new_code/code/runs/001cf4cc-8939v2l_noisy_Fine/fold_3/model_best.pth.tar',
    '/home/shivgup/siim/heng/code/ayush/new_code/code/runs/001cf4cc-8939v2l_noisy_Fine/fold_4/model_best.pth.tar',
    ]


#### YOUR TRAINING CONFIGS
CONFIGS = [
    '/home/shivgup/siim/heng/code/ayush/new_code/code/runs/001cf4cc-8939v2l_noisy_Fine/fold_0/config.yaml',
    '/home/shivgup/siim/heng/code/ayush/new_code/code/runs/001cf4cc-8939v2l_noisy_Fine/fold_1/config.yaml',
    '/home/shivgup/siim/heng/code/ayush/new_code/code/runs/001cf4cc-8939v2l_noisy_Fine/fold_2/config.yaml',
    '/home/shivgup/siim/heng/code/ayush/new_code/code/runs/001cf4cc-8939v2l_noisy_Fine/fold_3/config.yaml',
    '/home/shivgup/siim/heng/code/ayush/new_code/code/runs/001cf4cc-8939v2l_noisy_Fine/fold_4/config.yaml',
    ]


MAIN_CFG = get_cfg()
MAIN_CFG.merge_from_file(CONFIGS[0])
DF = pd.read_csv(MAIN_CFG.PATH_CSV)
DF_f = pd.read_csv(MAIN_CFG.PATH_CSV)
DF_f_r = pd.read_csv(MAIN_CFG.PATH_CSV)
DF.head()


In [11]:
models = []
dls = []

for i, c in enumerate(CHECKPOINTS):
    MAIN_CFG = get_cfg()
    MAIN_CFG.merge_from_file(CONFIGS[i])
    datamodule = LoadDatasets(MAIN_CFG, fold=i)
    datamodule.setup()
    dls.append(datamodule.val_dataloader())
    model = build_model(MAIN_CFG)
    load_checkpoint(model, c, strict=True)
    models.append(model)

In [12]:
@torch.no_grad()
def get_preds(model, test_loader):
    model.cuda()
    model.eval()
    
    batch_probs = []
    batch_ids   = []
    
    for i, (batch) in enumerate(tqdm(test_loader, dynamic_ncols=True)):
        images = batch['image']
        image_id = batch['image_id']
        images = images.cuda()
        y_preds= model(images)['output']
        
        batch_probs.append(F.softmax(y_preds))
        batch_ids.append(image_id)
    
    del model
    torch.cuda.empty_cache()
    gc.collect()
    return torch.cat(batch_probs).data.cpu().numpy(), np.hstack(batch_ids)

In [13]:
for i in tqdm(range(5), dynamic_ncols=True):
    probs, ids = get_preds(models[i], dls[i])
    for (image_id, pred) in zip(ids, probs):
        index = DF.loc[DF.id == image_id].index
        DF.loc[index, CLASS_NAMES] = pred

display_df(DF.head())

In [ ]:
DF.to_csv('soft.csv', index=False)

In [1]:
# from torchmetrics import AveragePrecision

# #DF = DF.sort_values(by='id').reset_index(drop=True)
# oof_labels = DF[CLASS_NAMES].values
# dframe = pd.read_csv(MAIN_CFG.PATH_CSV)
# #dframe = dframe[dframe['kfold']!=-1].reset_index(drop=True)
# #dframe = dframe.sort_values(by='id').reset_index(drop=True)
# ori_labels = dframe[CLASS_NAMES].values

# oof_labels = torch.as_tensor(oof_labels)

# ori_labels = torch.as_tensor(ori_labels)
# ori_labels = torch.argmax(ori_labels, dim=1).long()

# mm = AveragePrecision(num_classes=MAIN_CFG.INPUT.NUM_CLASSES)
# pc = mm(oof_labels, ori_labels)
# av = torch.mean(torch.stack(pc))*2/3

# print('Print Per Class mAP: ', pc)
# print('Average mAP', av)